In [ ]:
import tqdm

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from sklearn import preprocessing # LabelEncoder
from sklearn.metrics import mean_squared_error # if squared=False; RMSE

from EnvDL.core import *
from EnvDL.dna import *


In [ ]:
cache_path = '../nbs_artifacts/01.03_g2fc_prep_matrices/'
ensure_dir_path_exists(dir_path = cache_path)

In [ ]:
load_from = '../nbs_artifacts/01.02_g2fc_imputation/'

meta = pd.read_csv(load_from+'meta0.csv')
# meta['Date_Planted'] = meta['Date_Planted'].astype(int)
# meta['Date_Harvested'] = meta['Date_Harvested'].astype(int)
phno = pd.read_csv(load_from+'phno0.csv')
soil = pd.read_csv(load_from+'soil0.csv')
wthr = pd.read_csv(load_from+'wthr0.csv')
# wthrWide = pd.read_csv(load_from+'wthrWide0.csv')
cgmv = pd.read_csv(load_from+'cgmv0.csv')

In [ ]:
mask = ((phno.Yield_Mg_ha.notna())) # This used to allow for missing yield so long as they were in 
# 2022. Now that the 2022 data is available they should be excluded.
phno = phno.loc[mask, :].reset_index(drop = True)
phno = phno.loc[:, ['Env', 'Year', 'Hybrid', 'Yield_Mg_ha']]
phno = phno.drop_duplicates().reset_index(drop = True).copy()

# Data Prep

## Prep CVs

In [ ]:
# YMat[phno.Year == 2021]

## Indexing

### Filtered Phenotype and lookup
There are huge data savings to be had from keeping only deduplciated genomic data in memory in the GPU. 

In [ ]:
uniq_hybrids = list(set(phno.Hybrid))
uniq_hybrids[0:3]

['CG120/CG102', 'MO44_PHW65_0164/PHT69', 'PHKE6/PHB47']

In [ ]:
# check snp records and only keep phenotypes with snp data
snps_found = [exists_geno((taxa_to_filename(taxa = ith_hybrid))) for ith_hybrid in  uniq_hybrids]

In [ ]:
temp = pd.DataFrame(zip(uniq_hybrids, snps_found), columns=['Hybrid', 'SNPS'])
temp.loc[(temp.SNPS != True)]

,Hybrid,SNPS
98,MO44_PHW65_0252/LH195,False
172,MBS3785/MBS5618,False
222,PHW65_MOG_0115/PHT69,False
414,TX130_RP/LH185,False
422,LH198/LH123HT,False
...,...,...
4832,GEMN-0263/DK3IIH6,False
4922,GEMN-0264/DK3IIH6,False
4951,PHW65_MOG_0119/LH195,False
4960,MO44_PHW65_0349/PHT69,False


In [ ]:
import os

In [ ]:
print('Genotypes flagged as missing that exist with different capitalization:')
missing_hybrids = temp.loc[(temp.SNPS != True), ['Hybrid']]
# quick check that some of these aren't just differing in capitalization
missing_hybrids_list = [e.replace('/', '__').lower() for e in list(missing_hybrids.Hybrid)]
print(
    [e for e in [e.lower for e in os.listdir('../data/zma/g2fc/genotypes/snps/')] if e in missing_hybrids_list]
)

Genotypes flagged as missing that exist with different capitalization:
[]


In [ ]:
print(
    str(missing_hybrids.merge(phno, how = 'left').shape[0]
       )+' observations ('+str(round(100 * (missing_hybrids.merge(phno, how = 'left').shape[0]/phno.shape[0]), 3)
       )+'%) missing genotype.'
)

2729 observations (2.017%) missing genotype.


In [ ]:
# limit to phno with snp data
phno_geno = temp.loc[(temp.SNPS == True)].merge(phno, how = 'left')
phno_geno

,Hybrid,SNPS,Env,Year,Yield_Mg_ha
0,CG120/CG102,True,IAH1c_2014,2014,8.885659
1,CG120/CG102,True,IAH2_2014,2014,8.739672
2,CG120/CG102,True,IAH2_2014,2014,5.949948
3,CG120/CG102,True,MNH1_2014,2014,7.731308
4,CG120/CG102,True,MNH1_2014,2014,7.780678
...,...,...,...,...,...
132598,PHN11_PHW65_0494/PHT69,True,OHH1_2019,2019,12.946697
132599,PHN11_PHW65_0494/PHT69,True,ONH2_2019,2019,10.595273
132600,PHN11_PHW65_0494/PHT69,True,WIH1_2019,2019,13.264511
132601,PHN11_PHW65_0494/PHT69,True,WIH2_2019,2019,9.562105


In [ ]:
# restrict to observations with snp data
phno_geno = phno.merge(phno_geno.loc[:, ['Hybrid', 'SNPS']].drop_duplicates(), how  = 'left')
phno_geno = phno_geno.loc[(phno_geno.SNPS == True)].drop(columns = ['SNPS']).reset_index(drop=True).copy()
phno_geno

,Env,Year,Hybrid,Yield_Mg_ha
0,DEH1_2014,2014,M0088/LH185,5.721725
1,DEH1_2014,2014,M0143/LH185,11.338246
2,DEH1_2014,2014,M0003/LH185,6.540810
3,DEH1_2014,2014,M0035/LH185,10.366857
4,DEH1_2014,2014,M0052/LH185,10.908814
...,...,...,...,...
132598,WIH3_2022,2022,W10010_0337/LH244,11.975018
132599,WIH3_2022,2022,W10010_0346/LH244,12.971193
132600,WIH3_2022,2022,W10010_0358/LH244,13.499769
132601,WIH3_2022,2022,W10010_0381/LH244,10.831640


In [ ]:
assert phno_geno.Yield_Mg_ha.isna().sum() == 0

In [ ]:
# phno_geno.to_csv(cache_path+'phno_geno_filter.csv', index=False)

phno_geno = phno_geno.reset_index().rename(columns = {'index':'Phno_Idx'})

In [ ]:
Env_Idxs  = phno_geno.loc[:, ['Env']
                    ].drop_duplicates(
                    ).reset_index(drop = True
                    ).reset_index(
                    ).rename(columns = {'index':'Env_Idx'})
Geno_Idxs = phno_geno.loc[:, ['Hybrid']
                    ].drop_duplicates(
                    ).reset_index(drop = True
                    ).reset_index(
                    ).rename(columns = {'index':'Geno_Idx'})

In [ ]:
# This is key. I'm explictly adding an index column to fix frustrations jumping between sorted versions of the df.

phno_geno = phno_geno.merge(Env_Idxs).merge(Geno_Idxs).sort_values('Phno_Idx').reset_index(drop = True)

In [ ]:
phno_geno = phno_geno.loc[:, [ 'Env', 'Year', 'Hybrid', 'Yield_Mg_ha', 
                              'Phno_Idx', 'Env_Idx', 'Geno_Idx']]

phno_geno

,Env,Year,Hybrid,Yield_Mg_ha,Phno_Idx,Env_Idx,Geno_Idx
0,DEH1_2014,2014,M0088/LH185,5.721725,0,0,0
1,DEH1_2014,2014,M0143/LH185,11.338246,1,0,1
2,DEH1_2014,2014,M0003/LH185,6.540810,2,0,2
3,DEH1_2014,2014,M0035/LH185,10.366857,3,0,3
4,DEH1_2014,2014,M0052/LH185,10.908814,4,0,4
...,...,...,...,...,...,...,...
132598,WIH3_2022,2022,W10010_0337/LH244,11.975018,132598,235,4871
132599,WIH3_2022,2022,W10010_0346/LH244,12.971193,132599,235,4872
132600,WIH3_2022,2022,W10010_0358/LH244,13.499769,132600,235,4873
132601,WIH3_2022,2022,W10010_0381/LH244,10.831640,132601,235,4875


In [ ]:
phno_geno.to_csv(cache_path+'phno_geno.csv', index=False)

In [ ]:
# and overwrite phno to prevent incomplete data from creeping in.
phno = phno_geno.copy()

### Observation to Genotype

In [ ]:
uniq_geno = phno_geno.loc[:, ['Geno_Idx']].drop_duplicates().reset_index().rename(columns = {'index':'Is_Phno_Idx'})
obs_geno_lookup = phno_geno.loc[:, ['Phno_Idx', 'Geno_Idx']].merge(uniq_geno).sort_values('Phno_Idx').reset_index(drop = True).copy()
obs_geno_lookup

,Phno_Idx,Geno_Idx,Is_Phno_Idx
0,0,0,0
1,1,1,1
2,2,2,2
3,3,3,3
4,4,4,4
...,...,...,...
132598,132598,4871,122780
132599,132599,4872,122781
132600,132600,4873,122782
132601,132601,4875,122784


In [ ]:
# save out obs_geno_lookup
np.save(cache_path+'obs_geno_lookup.npy', np.asarray(obs_geno_lookup))
obs_geno_lookup.to_csv(cache_path+'obs_geno_lookup.csv', index=False)

### Observation to Environment

In [ ]:
uniq_env = phno_geno.loc[:, ['Env_Idx']].drop_duplicates().reset_index().rename(columns = {'index':'Is_Phno_Idx'})
obs_env_lookup = phno_geno.loc[:, ['Phno_Idx', 'Env_Idx']].merge(uniq_env).sort_values('Phno_Idx').reset_index(drop = True).copy()
obs_env_lookup

,Phno_Idx,Env_Idx,Is_Phno_Idx
0,0,0,0
1,1,0,0
2,2,0,0
3,3,0,0
4,4,0,0
...,...,...,...
132598,132598,235,132161
132599,132599,235,132161
132600,132600,235,132161
132601,132601,235,132161


In [ ]:
# save out obs_geno_lookup
np.save(cache_path+'obs_env_lookup.npy', np.asarray(obs_env_lookup))
obs_env_lookup.to_csv(cache_path+'obs_env_lookup.csv', index=False)

## Y

In [ ]:
YMat = np.array(phno_geno.Yield_Mg_ha)
np.save(cache_path+'YMat.npy', YMat)

In [ ]:




# phno_geno.drop_duplicates(
#                              ).reset_index(
#                              ).rename(columns = {'index':'Phno_Idx'})

In [ ]:
# produce the final lookup table

In [ ]:
# obs_lookup = phno_geno.loc[:, ['Hybrid']
#                              ].drop_duplicates(
#                              ).reset_index(
#                              ).rename(columns = {'index':'Is_Phno_Idx'}
#                              ).reset_index().rename(columns = {'index':'Geno_Idx'})
# # add in obs idx in phno
# obs_lookup = phno_geno.merge(obs_lookup).reset_index().rename(columns = {'index':'Phno_Idx'})
# obs_geno_lookup = obs_lookup.loc[: ,  ['Phno_Idx', 'Geno_Idx', 'Is_Phno_Idx']]
# obs_geno_lookup

In [ ]:
# obs_geno_lookup

In [ ]:
# # save out obs_geno_lookup
# np.save(cache_path+'obs_geno_lookup.npy', np.asarray(obs_geno_lookup))
# obs_geno_lookup.to_csv(cache_path+'obs_geno_lookup.csv', index=False)

## Genomic Data

### One Hot Encode G (Deduplicated)

In [ ]:
# If phno and GMat observations are 1:1, then GMat is 135793, 2250. After deduplicating this will become 4926, 2250. That's only 3.6%!
# Doing this removes several hybrids so it's actually a little smaller 4926, 2203

In [ ]:
obs_geno_lookup.head()

,Phno_Idx,Geno_Idx,Is_Phno_Idx
0,0,0,0
1,1,1,1
2,2,2,2
3,3,3,3
4,4,4,4


In [ ]:
temp = obs_geno_lookup.drop(columns = 'Phno_Idx').drop_duplicates().reset_index(drop=True)

mask = phno_geno.index.isin(list(obs_geno_lookup['Is_Phno_Idx'].drop_duplicates()))
temp = phno_geno.loc[mask, ]#'Hybrid']
temp = temp.reset_index(drop = True) # drop so that the index in the DataFrame and ndarray match
temp

,Env,Year,Hybrid,Yield_Mg_ha,Phno_Idx,Env_Idx,Geno_Idx
0,DEH1_2014,2014,M0088/LH185,5.721725,0,0,0
1,DEH1_2014,2014,M0143/LH185,11.338246,1,0,1
2,DEH1_2014,2014,M0003/LH185,6.540810,2,0,2
3,DEH1_2014,2014,M0035/LH185,10.366857,3,0,3
4,DEH1_2014,2014,M0052/LH185,10.908814,4,0,4
...,...,...,...,...,...,...,...
4921,IAH2_2022,2022,GEMN-0096_LH212HT_0014/LH244,10.929844,124228,217,4921
4922,IAH2_2022,2022,NC370/LH244,5.306059,124582,217,4922
4923,IAH3_2022,2022,GEMN-0097_LH212HT_0046/LH244,13.179435,124819,218,4923
4924,IAH4_2022,2022,GEMN-0225_PHK76_0025/LH244,11.453161,125555,219,4924


In [ ]:
temp = pd.concat([temp, temp.Hybrid.str.split('/', expand=True)], axis=1
        ).rename(columns = {0:'P0', 1:'P1'})

uniq_parents = list(set(pd.concat([temp['P0'], temp['P1']])))

In [ ]:
if os.path.exists(cache_path+'GMat.npy'):
    GMat = np.load(cache_path+'GMat.npy')
else:
    GMat = np.zeros([temp.shape[0], len(uniq_parents)])

    for j in tqdm.tqdm(range(len(uniq_parents))):
        for parent in ['P0', 'P1']:
            mask = (temp[parent] == uniq_parents[j]) 
            GMat[temp.loc[mask, ].index, j] += 1
            
    np.save(cache_path+'GMat.npy', GMat)

In [ ]:
np.save(cache_path+'GMatNames.npy', uniq_parents)

In [ ]:
# confirm there are two parents encoded for each observation
assert 2 == np.min(np.sum(GMat, axis = 1))

### Nucleotides

In [ ]:
# Convert IUPAC codes into one hot
# https://www.bioinformatics.org/sms/iupac.html
IUPAC = ['unk', 'A', 'C', 'G', 'T', 'K', 'M', 'N', 'R', 'S', 'W', 'Y']
IUPAC = dict(zip(IUPAC, [i for i in range(len(IUPAC))])) # begin with unk.
IUPAC

{'unk': 0,
 'A': 1,
 'C': 2,
 'G': 3,
 'T': 4,
 'K': 5,
 'M': 6,
 'N': 7,
 'R': 8,
 'S': 9,
 'W': 10,
 'Y': 11}

In [ ]:
put_cached_result(cache_path+'ACGT_OneHot_dict.pkl', IUPAC)

In [ ]:
if os.path.exists(cache_path+'ACGT_OneHot.npy'):
#     ACGT = np.load(cache_path+'ACGT_OneHot.npy')
    pass
else:
    temp = obs_geno_lookup.drop(columns = 'Phno_Idx').drop_duplicates().reset_index(drop=True)

    # use the first Hybrid to figure out what the length of the sequence is.
    i = 0
    phno_idx = temp.loc[i, 'Is_Phno_Idx']
    ith_hybrid = phno.loc[phno_idx, 'Hybrid']
    geno_seq_len = len(get_geno(taxa_to_filename(taxa = ith_hybrid))[1:])    

    # setup ndarray to hold data
    ACGT = np.ndarray(shape = (temp.shape[0],        # obs
                               geno_seq_len,         # values
                               len(IUPAC.keys())+1)) # channels

    # This is inefficient but needs only to be run once.
    # go over observations, channels, fill in values

    for i in tqdm.tqdm(temp.index): 
        phno_idx = temp.loc[i, 'Is_Phno_Idx']
        ith_hybrid = phno.loc[phno_idx, 'Hybrid']
        res = get_geno(taxa_to_filename(taxa = ith_hybrid)) 
        res = res[1:] # drop taxa

        res = [e.strip('\n') for e in res] # remove end of line

        for key in IUPAC.keys():
            IUPAC_mask = [True if e == key else False for e in res]
            ACGT[i, 
                 IUPAC_mask, 
                 IUPAC[key]] = 1  

    # Swap axes to match pytorch convention
    ACGT = np.swapaxes(ACGT, 1, 2)

    np.save(cache_path+'ACGT_OneHot.npy', ACGT)

### Nucleotide percents

In [ ]:
if os.path.exists(cache_path+'ACGT.npy'):
#     ACGT = np.load(cache_path+'ACGT.npy')
    pass
else:
    temp = obs_geno_lookup.drop(columns = 'Phno_Idx').drop_duplicates().reset_index(drop=True)

    
    # use the first Hybrid to figure out what the length of the sequence is.
    i = 0
    phno_idx = temp.loc[i, 'Is_Phno_Idx']
    ith_hybrid = phno.loc[phno_idx, 'Hybrid']
    geno_seq_len = len(get_geno(taxa_to_filename(taxa = ith_hybrid))[1:])    
    
    # setup ndarray to hold data
    ACGT = np.ndarray(shape = (temp.shape[0], geno_seq_len, 4))

    for i in tqdm.tqdm(temp.index):
        phno_idx = temp.loc[i, 'Is_Phno_Idx']
        ith_hybrid = phno.loc[phno_idx, 'Hybrid']
        res = get_geno(taxa_to_filename(taxa = ith_hybrid)) 
        res = res[1:] # drop taxa
        res = list_to_ACGT(in_seq = res)
        ACGT[i, :, :] = res[None, :, :]
        
    # Swap axes to match pytorch convention
    # (4926, 125891, 4)
    ACGT = np.swapaxes(ACGT, 1, 2)

    # set missings to 0 
    ACGT[np.isnan(ACGT)] = 0

    np.save(cache_path+'ACGT.npy', ACGT)

### Hilbert Nucleotide percents

In [ ]:
if os.path.exists(cache_path+'ACGT_hilb.npy'):
#     ACGT_hilb = np.load(cache_path+'ACGT_hilb.npy')
    pass
else:
    ACGT_hilb = np_3d_to_hilbert( np.swapaxes(ACGT, 1, 2) ) # swap channels back to dim 2 before running

    # ACGT_hilb.shape
    # (4926, 256, 512, 4)
    # Pytorch standard has channels second
    ACGT_hilb = np.swapaxes(ACGT_hilb, 1, 3)
    ACGT_hilb = np.swapaxes(ACGT_hilb, 2, 3)

    # set missings to 0
    ACGT_hilb[np.isnan(ACGT_hilb)] = 0

    np.save(cache_path+'ACGT_hilb.npy', ACGT_hilb)

## Environmental Data

### Make Environment to Observation Lookup 

In [ ]:
# obs_geno_lookup

In [ ]:
# phno_geno

In [ ]:
# # working from phno_geno to ensure there is genetic data.
# uniq_Env = phno_geno.loc[:, ['Env']].drop_duplicates(
#                                    ).reset_index().rename(columns = {'index': 'Is_Phno_Idx'}
#                                    ).reset_index().rename(columns = {'index': 'Is_Env_Idx'})
# uniq_Env

In [ ]:
# obs_env_lookup = phno_geno.merge(uniq_Env
#                          ).reset_index().rename(columns = {'index': 'Phno_Idx'}
#                          ).loc[:, ['Phno_Idx', 'Is_Env_Idx', 'Is_Phno_Idx']]
# obs_env_lookup

In [ ]:
# # confrim obs_env_lookup is set the way it should be.
# temp = pd.concat([phno_geno.drop(columns = 'SNPS').reset_index(drop = True), # have to reset to correctly cbind these
#                   obs_env_lookup], 
#                  axis=1)

# temp = temp.loc[:, ['Env', 'Is_Env_Idx']].drop_duplicates(
#                                         ).assign(n = 1
#                                         ).groupby(['Env'#, 'Is_Env_Idx'
#                                                   ]
#                                         ).count(
#                                         ).reset_index()

# temp

In [ ]:
# assert 1 == max(temp.n)

In [ ]:
# # save out obs_geno_lookup
# np.save(cache_path+'obs_env_lookup.npy', np.asarray(obs_env_lookup))
# obs_env_lookup.to_csv(cache_path+'obs_env_lookup.csv', index=False)

In [ ]:
# # useful for making S, W Matricies
# obs_env_lookup_small = obs_env_lookup.drop(columns = 'Phno_Idx').drop_duplicates().sort_values('Is_Env_Idx').reset_index(drop = True)

# obs_env_lookup_small

In [ ]:
obs_env_lookup_small = obs_env_lookup.drop(columns = 'Phno_Idx'
                                    ).drop_duplicates(
                                    ).sort_values('Env_Idx'
                                    ).reset_index(drop = True)

obs_env_lookup_small.head()

,Env_Idx,Is_Phno_Idx
0,0,0
1,1,461
2,2,936
3,3,1785
4,4,2222


### Make S Matrix

In [ ]:
SMat = phno.loc[:, ['Env']].merge(soil.drop(columns = ['Unnamed: 0', 'Year'])).drop(columns = ['Env'])
SMatNames = list(SMat)
SMat = np.array(SMat)

In [ ]:
SMat.shape

(132603, 23)

In [ ]:
SMatSmall = np.zeros([
    obs_env_lookup_small.shape[0], # unique environments
    len(SMatNames)                 # Cin
])

In [ ]:
for i in obs_env_lookup_small.index:
    ii = obs_env_lookup_small.loc[(obs_env_lookup_small.Env_Idx == i), 'Is_Phno_Idx']
    ii = int(ii)
    SMatSmall[i, ] = SMat[ii, ]

In [ ]:
# overwrite with much smaller version (rows from 135793 -> 236)
SMat = SMatSmall.copy()

### Make W Matrix

In [ ]:
# Input: (N,Cin,Lin)(N,Cin,Lin) or (Cin,Lin)(Cin,Lin)

In [ ]:
WMatNames = list(wthr.drop(columns = ['Unnamed: 0', 'Env', 'Year', 'Date', 'DOY']))
WMat = np.zeros([                  # Pytorch uses
    obs_env_lookup_small.shape[0], # N
    len(WMatNames),                # Cin
    np.max(wthr.DOY)               # Lin
])

In [ ]:
for i in tqdm.tqdm(obs_env_lookup_small.index):
    ii = obs_env_lookup_small.loc[(obs_env_lookup_small.Env_Idx == i), 'Is_Phno_Idx']
    ii = int(ii)

    ith_Env = phno_geno.loc[(phno_geno.index == ii), 'Env']
    ith_Env = list(ith_Env)[0]

    # selected data is transposed to match correct shape
    wthr_mask = (wthr.Env == ith_Env)
    WMat[i, :, :] = wthr.loc[wthr_mask, 
                       ].sort_values('DOY'
                       ).drop(columns = ['Unnamed: 0', 'Env', 
                                         'Year', 'Date', 'DOY']).T

100%|██████████████████████████████████████████████████████████████████████████████████| 236/236 [00:00<00:00, 288.29it/s]


In [ ]:
# WMatNames = list(wthr.drop(columns = ['Unnamed: 0', 'Env', 'Year', 'Date', 'DOY']))
# WMat = np.zeros([   # Pytorch uses
#     phno.shape[0],  # N
#     len(WMatNames), # Cin
#     np.max(wthr.DOY)# Lin
# ])

In [ ]:
# # loop through all obs, but only add each env once (add to all relevant obs)
# added_envs = []
# for i in tqdm.tqdm(phno.index):
#     env = phno.loc[i, 'Env']

#     if env in added_envs:
#         pass
#     else:
#         mask = (phno.Env == env)
#         WMat_idxs = phno.loc[mask, ].index

#         # selected data is transposed to match correct shape
#         wthr_mask = (wthr.Env == env)
#         WMat[WMat_idxs, :, :] = wthr.loc[wthr_mask, 
#                                    ].sort_values('DOY'
#                                    ).drop(columns = ['Unnamed: 0', 'Env', 
#                                                      'Year', 'Date', 'DOY']).T

#         added_envs += [env]

## Prep CGMV?

In [ ]:
# MMatNames = list(cgmv.drop(columns = ['Unnamed: 0', 'Env', 'Year']))

In [ ]:
# MMat = np.zeros([   
#     phno.shape[0],  
#     len(MMatNames)
# ])

In [ ]:
# # loop through all obs, but only add each env once (add to all relevant obs)
# added_envs = []
# for i in tqdm.tqdm(phno.index):
#     env = phno.loc[i, 'Env']

#     if env in added_envs:
#         pass
#     else:
#         mask = (phno.Env == env)
#         MMat_idxs = phno.loc[mask, ].index

#         # selected data is transposed to match correct shape
#         cgmv_mask = (cgmv.Env == env)
#         MMat[MMat_idxs, :] = cgmv.loc[cgmv_mask, 
#                                 ].drop(columns = ['Unnamed: 0', 'Env', 'Year'])

#         added_envs += [env]

In [ ]:
phno_geno.loc[:, ['Env', 'Env_Idx']].merge(cgmv)

Env  Env_Idx  Unnamed: 0  Year  SDR_pGerEme_1  SDR_pEmeEnJ_1  \
0       DEH1_2014        0           0  2014       0.966035       1.615324   
1       DEH1_2014        0           0  2014       0.966035       1.615324   
2       DEH1_2014        0           0  2014       0.966035       1.615324   
3       DEH1_2014        0           0  2014       0.966035       1.615324   
4       DEH1_2014        0           0  2014       0.966035       1.615324   
...           ...      ...         ...   ...            ...            ...   
132598  WIH3_2022      235         238  2022       3.565136       1.762135   
132599  WIH3_2022      235         238  2022       3.565136       1.762135   
132600  WIH3_2022      235         238  2022       3.565136       1.762135   
132601  WIH3_2022      235         238  2022       3.565136       1.762135   
132602  WIH3_2022      235         238  2022       3.565136       1.762135   

        SDR_pEnJFlo_1  SDR_pFloFla_1  SDR_pFlaFlw_1  SDR_pFlwStG_1  \
0            0.542019       0.317045       0.617058       0.773524   
1            0.542019       0.317045       0.617058       0.773524   
2            0.542019       0.317045       0.617058       0.773524   
3            0.542019       0.317045       0.617058       0.773524   
4            0.542019       0.317045       0.617058       0.773524   
...               ...            ...            ...            ...   
132598       0.441583       0.635249      -0.025162       0.746584   
132599       0.441583       0.635249      -0.025162       0.746584   
132600       0.441583       0.635249      -0.025162       0.746584   
132601       0.441583       0.635249      -0.025162       0.746584   
132602       0.441583       0.635249      -0.025162       0.746584   

        SDR_pStGEnG_1  SDR_pEnGMat_1  SDR_pMatHar_1  SDR_pGerEme_2  \
0            0.983128       0.407474       0.350057       1.524351   
1            0.983128       0.407474       0.350057       1.524351   
2            0.983128       0.407474       0.350057       1.524351   
3            0.983128       0.407474       0.350057       1.524351   
4            0.983128       0.407474       0.350057       1.524351   
...               ...            ...            ...            ...   
132598       2.090461       2.193082       1.430046       3.009535   
132599       2.090461       2.193082       1.430046       3.009535   
132600       2.090461       2.193082       1.430046       3.009535   
132601       2.090461       2.193082       1.430046       3.009535   
132602       2.090461       2.193082       1.430046       3.009535   

        SDR_pEmeEnJ_2  SDR_pEnJFlo_2  SDR_pFloFla_2  SDR_pFlaFlw_2  \
0            1.798886       0.919594       0.571503       0.326560   
1            1.798886       0.919594       0.571503       0.326560   
2            1.798886       0.919594       0.571503       0.326560   
3            1.798886       0.919594       0.571503       0.326560   
4            1.798886       0.919594       0.571503       0.326560   
...               ...            ...            ...            ...   
132598       2.262823       0.710480       0.768372       0.296266   
132599       2.262823       0.710480       0.768372       0.296266   
132600       2.262823       0.710480       0.768372       0.296266   
132601       2.262823       0.710480       0.768372       0.296266   
132602       2.262823       0.710480       0.768372       0.296266   

        SDR_pFlwStG_2  SDR_pStGEnG_2  SDR_pEnGMat_2  SDR_pMatHar_2  \
0            0.545948       1.051211       1.247855       1.228458   
1            0.545948       1.051211       1.247855       1.228458   
2            0.545948       1.051211       1.247855       1.228458   
3            0.545948       1.051211       1.247855       1.228458   
4            0.545948       1.051211       1.247855       1.228458   
...               ...            ...            ...            ...   
132598       0.603923       1.930914       2.472133       2.0164

In [ ]:
cgmv_Env = uniq_Env.merge(cgmv)
cgmv_Env = cgmv_Env.drop(columns = ['Is_Env_Idx', 'Is_Phno_Idx', 'Env', 'Unnamed: 0', 'Year'])
MMatNames = list(cgmv_Env)
MMat = np.asarray(cgmv_Env)

NameError: name 'uniq_Env' is not defined

# Save data
This will streamline model generation. I'll just need to load these files in and can directly begin modeling.

In [ ]:
np.save(cache_path+'SMatNames.npy', SMatNames)
np.save(cache_path+'WMatNames.npy', WMatNames)
np.save(cache_path+'MMatNames.npy', MMatNames)

In [ ]:
# phno.to_csv(cache_path+'phno.csv', index=False)

In [ ]:
np.save(cache_path+'SMat.npy', SMat)
np.save(cache_path+'WMat.npy', WMat)
np.save(cache_path+'MMat.npy', MMat)